## Summary

## Install dependencies

In [ ]:
try:
    import google.colab
    GOOGLE_COLAB = True
except ImportError:
    GOOGLE_COLAB = False

In [ ]:
if GOOGLE_COLAB:
    !pip install "git+https://github.com/ostrokach/beam.git@develop#egg=apache_beam[gcp]&subdirectory=sdks/python"

## Imports

In [ ]:
import itertools
import tempfile

import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.runners.interactive.cache_manager import CacheManager

In [ ]:
beam.__version__

In [ ]:
%matplotlib inline

## Parameters

In [ ]:
NOTEBOOK_NAME = "demo_explicit_caching"

In [ ]:
temp_dir = tempfile.mkdtemp(prefix="beam-temp-")
temp_dir

In [ ]:
options = PipelineOptions(runner="DirectRunner", temp_location=temp_dir)
print(options.display_data())

In [ ]:
cache_manager = CacheManager(options)

## Workflow

### Populate input cache

In [ ]:
input_cache = cache_manager.create_cache_from_defaults()

with beam.Pipeline(options=options) as p:
    _ = (
        #
        p
        | beam.Create(list(range(10)))
        | input_cache.writer()
    )

In [ ]:
for i in itertools.islice(input_cache.read(), 5):
    print(i)

### Squares

In [ ]:
squares_cache = cache_manager.create_cache_from_defaults()

with beam.Pipeline(options=options) as p:
    squares = (
        #
        p
        | input_cache.reader()
        | beam.Map(lambda e: e ** 2)
        | squares_cache.writer()
    )

In [ ]:
for i in itertools.islice(squares_cache.read(), 5):
    print(i)

<span style="color: blue">Should we add `.reader()` as the default `.expand()` for cache?</span>

### Cubes

In [ ]:
cubes_cache = cache_manager.create_cache_from_defaults()

with beam.Pipeline(options=options) as p:
    squares = (
        #
        p
        | input_cache.reader()
        | beam.Map(lambda e: e ** 3)
        | cubes_cache.writer()
    )

In [ ]:
for i in itertools.islice(cubes_cache.read(), 5):
    print(i)

### Plot

In [ ]:
init_list = list(range(10))
squares_list = sorted(squares_cache.read())
cubes_list = sorted(cubes_cache.read())

from matplotlib import pyplot as plt
plt.scatter(init_list, squares_list, label='squares', color='red')
plt.scatter(init_list, cubes_list, label='cubes', color='blue')
plt.legend(loc='upper left')
plt.show()